### Exploration

In [1]:
!git clone https://github.com/openai/baselines.git
%cd baselines
!pip install -e .
!pip install stable-baselines
!pip install --upgrade tensorflow==1.14.0
!pip install gym
!pip install pyarrow

fatal: destination path 'baselines' already exists and is not an empty directory.
/project/baselines
Obtaining file:///project/baselines
  Attempting uninstall: baselines
    Found existing installation: baselines 0.1.6
    Uninstalling baselines-0.1.6:
      Successfully uninstalled baselines-0.1.6
  Running setup.py develop for baselines
Requirement already up-to-date: tensorflow==1.14.0 in /opt/conda/lib/python3.7/site-packages (1.14.0)


In [2]:
import os
import time
import gym
import random
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

from stable_baselines.deepq.policies import MlpPolicy
from stable_baselines.common.vec_env.dummy_vec_env import DummyVecEnv

from stable_baselines import deepq
from stable_baselines import bench
from stable_baselines import logger
import tensorflow as tf

from baselines.common.tf_util import make_session

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [3]:
cicddos2019 = pd.read_feather("/project/clean-ids-collection/cic-ddos2019/clean/cicddos2019.feather")
cicdos2017 = pd.read_feather("/project/clean-ids-collection/cic-dos2017/clean/cicdos2017.feather")
cicids2017 = pd.read_feather("/project/clean-ids-collection/cic-ids2017/clean/cicids2017.feather")
data = pd.concat([cicddos2019, cicdos2017 ,cicids2017], ignore_index=True)
# data = pd.read_feather("/project/clean-ids-collection/cic-dos2017/clean/cicdos2017.feather")
data.head(5)

,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,2018-11-03 09:18:16.964447,114456999,45,0,0.0,0.0,0.0,0.0,0.000000,...,0,8185.583496,28337.111328,98168.0,3.0,9529897.0,351582.62500,10001143.0,9048097.0,Benign
1,0,2018-11-03 09:18:18.506537,114347504,56,0,0.0,0.0,0.0,0.0,0.000000,...,0,35028.417969,121314.914062,420255.0,4.0,9493930.0,351541.09375,9978130.0,8820294.0,Benign
2,6,2018-11-03 09:18:18.610576,36435473,6,2,116.0,92.0,46.0,6.0,19.333334,...,20,62416.000000,0.000000,62416.0,62416.0,36373056.0,0.00000,36373056.0,36373056.0,Benign
3,6,2018-11-03 09:18:18.610579,36434705,6,2,116.0,92.0,46.0,6.0,19.333334,...,20,62413.000000,0.000000,62413.0,62413.0,36372292.0,0.00000,36372292.0,36372292.0,Benign
4,6,2018-11-03 09:18:18.610581,36434626,6,2,116.0,92.0,46.0,6.0,19.333334,...,20,62409.000000,0.000000,62409.0,62409.0,36372216.0,0.00000,36372216.0,36372216.0,Benign


In [4]:
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2558780 entries, 0 to 2558779
Data columns (total 69 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   Protocol                  int8          
 1   Timestamp                 datetime64[ns]
 2   Flow Duration             int32         
 3   Total Fwd Packets         int32         
 4   Total Backward Packets    int32         
 5   Fwd Packets Length Total  float64       
 6   Bwd Packets Length Total  float64       
 7   Fwd Packet Length Max     float32       
 8   Fwd Packet Length Min     float32       
 9   Fwd Packet Length Mean    float64       
 10  Fwd Packet Length Std     float64       
 11  Bwd Packet Length Max     float32       
 12  Bwd Packet Length Min     float32       
 13  Bwd Packet Length Mean    float64       
 14  Bwd Packet Length Std     float64       
 15  Flow Bytes/s              float64       
 16  Flow Packets/s            float64       
 17  Flow IAT

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Act Data Packets,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,2.558780e+06,2.558780e+06,2.558780e+06,2.558780e+06,2.558780e+06,2.558780e+06,2.558780e+06,2.558780e+06,2.558780e+06,2.558780e+06,...,2.558780e+06,2.558780e+06,2.558780e+06,2.558780e+06,2.558780e+06,2.558780e+06,2.558780e+06,2.558780e+06,2.558780e+06,2.558780e+06
mean,1.030261e+01,1.822769e+07,1.153743e+01,1.312536e+01,8.958786e+02,1.967872e+04,2.442396e+02,2.622631e+01,7.261834e+01,7.971624e+01,...,6.636542e+00,-6.372257e+05,1.116707e+05,5.248293e+04,1.990886e+05,8.119350e+04,9.749128e+06,6.108299e+05,1.022345e+07,9.266053e+06
std,5.381759e+00,3.625484e+07,7.895447e+02,1.051686e+03,2.153483e+04,2.382625e+06,7.560385e+02,8.891830e+01,2.054616e+02,2.952703e+02,...,6.696438e+02,2.598903e+07,8.144914e+05,4.717359e+05,1.224279e+06,7.278005e+05,2.513104e+07,4.928044e+06,2.591850e+07,2.486494e+07
min,0.000000e+00,-1.300000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,-1.408238e+09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.000000e+00,1.029500e+04,2.000000e+00,1.000000e+00,3.000000e+01,6.000000e+00,2.000000e+01,0.000000e+00,7.000000e+00,0.000000e+00,...,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,6.000000e+00,9.191950e+04,2.000000e+00,2.000000e+00,7.400000e+01,1.840000e+02,4.400000e+01,0.000000e+00,3.800000e+01,0.000000e+00,...,1.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.700000e+01,6.142980e+06,6.000000e+00,6.000000e+00,3.620000e+02,2.279000e+03,2.740000e+02,3.800000e+01,5.400000e+01,7.805031e+01,...,3.000000e+00,3.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.700000e+01,1.200000e+08,2.197590e+05,2.919220e+05,1.526642e+07,6.554530e+08,3.212000e+04,4.030000e+03,5.940857e+03,7.125597e+03,...,2.135570e+05,1.480000e+03,1.100000e+08,7.420000e+07,1.100000e+08,1.100000e+08,1.200000e+08,7.690000e+07,1.200000e+08,1.200000e+08


Preprocessing done by Laurens D'Hooge

## Creating the model
- Build an agent to classify network flow automatically
- Feed a packet that gets classified
- Want the classification to be equal to the label

In [5]:
def balancing_dataset_sampler_df(df, train_frac=0.2, val_frac=0.1, test_frac=0.7):
    col = df.columns[-1]
    print(col)
    cols = df.columns[:-1]
    print(cols)
    vc = df[col].value_counts()
    print(vc)
    n = vc.iloc[-1]
    print(n)
    m = vc.iloc[0]
    print(m)
    print(int(m-n))
    initial_cut = df.loc[df[col] == vc.index[0]].sample(n=int(m-n), replace=False)
    print(initial_cut.index)
    df = df.drop(index=initial_cut.index)
    vc = df[col].value_counts()
    print(vc)
    print(int(n*train_frac))
    train_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*train_frac), replace=False))
    train_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=train_df.index)

    validation_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*val_frac), replace=False))
    validation_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=validation_df.index)

    test_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*test_frac), replace=False))
    test_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=test_df.index)

    return train_df[cols], train_df[col], validation_df[cols], validation_df[col], test_df[cols], test_df[col]


In [6]:
data.drop('Timestamp', inplace=True, axis=1)
print(data['Label'].value_counts())
data['Label'] = data['Label'].astype('object')
atk_idx = data.loc[data['Label'] != "Benign"].index
data.loc[atk_idx, 'Label'] = 1.0
data.loc[data.index.difference(atk_idx), 'Label'] = 0.0
data['Label'] = data['Label'].astype(dtype=np.float32)



Benign                          2165849
DoS Hulk                         172778
DDoS                             128011
DrDoS_NTP                         13687
TFTP                              13027
DoS GoldenEye                     10286
Syn                                6713
FTP-Patator                        5931
DoS slowloris                      5383
DoS Slowhttptest                   5228
ddossim                            4991
SSH-Patator                        3219
slowread                           2641
UDP                                2158
slowloris                          1850
PortScan                           1850
MSSQL                              1763
slowheaders                        1649
Web Attack ï¿½ Brute Force         1470
Bot                                1400
DrDoS_UDP                          1174
UDP-lag                            1097
DrDoS_MSSQL                         927
hulk                                714
rudy                                692


In [7]:
x_train, y_train, x_val, y_val, x_test, y_test  = balancing_dataset_sampler_df(data, train_frac=0.9, val_frac=0.0, test_frac=0.1)

Label
Index(['Protocol', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Fwd Packets Length Total',
       'Bwd Packets Length Total', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
      

In [8]:
print("Training")
print(x_train.shape)
print(y_train.shape)
print("Validation")
print(x_val.shape)
print(y_val.shape)
print("Testing")
print(x_test.shape)
print(y_test.shape)


Training
(707274, 67)
(707274,)
Validation
(0, 67)
(0,)
Testing
(78586, 67)
(78586,)


In [9]:
# all attacks
# attacks_dict = {
#     'Benign': 0,
#     'Portmap': 1,
#     'NetBIOS': 2,
#     'LDAP': 3,
#     'MSSQL': 4,
#     'UDP': 5,
#     'UDPLag': 6,
#     'Syn': 7,
#     'DrDoS_NTP': 8,
#     'DrDoS_DNS': 9,
#     'DrDoS_LDAP': 10,
#     'DrDoS_MSSQL': 11,
#     'DrDoS_NetBIOS': 12,
#     'DrDoS_SNMP': 13,
#     'DrDoS_SSDP': 14,
#     'DrDoS_UDP': 15,
#     'UDP-lag': 16,
#     'WebDDoS': 17,
#     'TFTP': 18,
# }


In [10]:
# x_train = np.expand_dims(x_train, -1)
# x_test = np.expand_dims(x_test, -1)
# x_val = np.expand_dims(x_val, -1)

In [11]:
# y_train = np.expand_dims(y_train, -1)
# y_test = np.expand_dims(y_test, -1)

In [12]:
print(x_train.shape)

(707274, 67)


In [13]:
print(x_train.index)
print(y_train.index)
y_train = y_train.to_frame()
y_test = y_test.to_frame()

Int64Index([ 849251,   69292,   55814,  927561, 2043431,  163332, 2172652,
             980350,  140624,   20338,
            ...
            1495451, 1309692, 1068171,   89296, 1367238, 1274997, 1318453,
            2523114, 2488421, 1487197],
           dtype='int64', length=707274)
Int64Index([ 849251,   69292,   55814,  927561, 2043431,  163332, 2172652,
             980350,  140624,   20338,
            ...
            1495451, 1309692, 1068171,   89296, 1367238, 1274997, 1318453,
            2523114, 2488421, 1487197],
           dtype='int64', length=707274)


In [14]:
# custom keys -> replace by index

x_train = x_train.set_index([pd.Index(range (0, len(x_train)))])
y_train = y_train.set_index([pd.Index(range (0, len(y_train)))])
x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])


In [15]:
print(x_train.index)
print(y_train.index)

RangeIndex(start=0, stop=707274, step=1)
RangeIndex(start=0, stop=707274, step=1)


In [16]:
print(y_train)

        Label
0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
...       ...
707269    1.0
707270    1.0
707271    1.0
707272    1.0
707273    1.0

[707274 rows x 1 columns]


In [17]:
print(y_train.iloc[17162,:])

Label    0.0
Name: 17162, dtype: float32


In [18]:
print(y_train.index)
print(x_train.index)

RangeIndex(start=0, stop=707274, step=1)
RangeIndex(start=0, stop=707274, step=1)


### Custom environment

In [19]:
class IdsEnv(gym.Env):
    def __init__(self, images_per_episode=1, dataset=(x_train, y_train), random=True):
        # Actions we can take, classify as malicious or non-malicious (later also the correct attack)
        # change to 19 if detectiong all different attacks
        self.action_space = gym.spaces.Discrete(2)
        # All the features we have, 67 features and 1 label. Label should not be included
        self.observation_space = gym.spaces.Box(low=float('-inf'), high=float('inf'), shape=(67,))
        self.images_per_episode = images_per_episode
        self.step_count = 0

        self.x, self.y = dataset
        self.random = random
        self.dataset_idx = 0
    
    def step(self, action):
        done = False
        reward = int(action == self.expected_action)

        obs = self._next_obs()

        self.step_count += 1
        if self.step_count >= self.images_per_episode:
            done = True

        return obs, reward, done, {}

    def _next_obs(self):
        if self.random:
            next_obs_idx = random.randint(0, len(self.x) - 1)
            self.expected_action = int(self.y.iloc[next_obs_idx,:])
            obs = self.x.iloc[next_obs_idx,:]

        else:
            obs = self.x.iloc[self.dataset_idx]
            self.expected_action = int(self.y.iloc[self.dataset_idx])

            self.dataset_idx += 1
            if self.dataset_idx >= len(self.x):
                raise StopIteration()

        return obs
    
    def reset(self):
        self.step_count = 0

        obs = self._next_obs()
        return obs

Train classifier using dqn

In [20]:
def ids_dqn():
#     logger.configure(dir=os.path.join(os.getcwd(),'Training/Logs/ids_dqn'), format_strs=['stdout', 'tensorboard'])
    env = IdsEnv(images_per_episode=4096)
    env = bench.Monitor(env, logger.get_dir())

    model = deepq.DQN(
        MlpPolicy, 
        env, 
        verbose=1, 
        learning_rate=0.00025, 
        buffer_size=1000000, 
        learning_starts=50000, 
        batch_size=32, 
        gamma=0.99, 
        train_freq=4, 
        target_network_update_freq=30000, # 10k to 30k
        exploration_fraction=0.01, # 0.1 to 0.01
        exploration_initial_eps=1.0, 
        exploration_final_eps=0.05, 
    ).learn(
        total_timesteps=int(1.0e6),
        log_interval=10
    )

    model.save('ddqn_stableb2_ids.pkl')
    env.close()
    
    return model

start_time = time.time()
dqn_model = ids_dqn()
print("DQN Training Time:", time.time() - start_time)








Instructions for updating:
Use keras.layers.flatten instead.


/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 10       |
| mean 100 episode reward | 2.05e+03 |
| steps                   | 36864    |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 20       |
| mean 100 episode reward | 1.94e+03 |
| steps                   | 77824    |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 30       |
| mean 100 episode reward | 1.82e+03 |
| steps                   | 118784   |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 40       |
| mean 100 episode reward | 1.77e+03 |
| steps                   | 159744   |
-------

KeyboardInterrupt: 

In [ ]:
def ids_dqn_eval(dqn_model):
    attempts, correct = 0,0

    env = IdsEnv(images_per_episode=1, dataset=(x_test, y_test), random=False)
    
    try:
        while True:
            obs, done = env.reset(), False
            while not done:
                obs, rew, done, _ = env.step(dqn_model.predict(obs)[0])

                attempts += 1
                if rew > 0:
                    correct += 1

    except StopIteration:
        print()
        print('validation done...')
        print('Accuracy: {0}%'.format((float(correct) / attempts) * 100))

ids_dqn_eval(dqn_model)

##### 